In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
from fastai.vision.all import *

In [ ]:
train_df = pd.read_csv(r'../input/plant-pathology-2020-fgvc7/train.csv')

In [ ]:
train_df.head(10)

In [ ]:
train_df.loc[(train['multiple_diseases']==1) & (train['scab']==1)]

In [ ]:
path = Path(r'../input/plant-pathology-2020-fgvc7')
Path.BASE_PATH = path

In [ ]:
(path/'images').ls()

In [ ]:
dblock = DataBlock()

In [ ]:
dsets = dblock.datasets(train_df)

In [ ]:
len(dsets.train), len(dsets.valid)

In [ ]:
#fname = (path/'images').ls()[0]
#re.findall(r'(.*)_\d+.jpg$',fname.name)

In [ ]:
x,y = dsets.train[0]
x,y

In [ ]:
x['image_id']

In [ ]:
train_df['image_id'] = train_df['image_id'] + '.jpg'
train_df['image_id'].astype('str')

In [ ]:
train_df.head()

In [ ]:
def reverse_one(row):
    for c in train_df.columns:
        if row[c]==1:
            return c
train_df['labels'] = train_df.apply(reverse_one, axis=1)

In [ ]:
train_df.head()

In [ ]:
dblock = DataBlock(get_x = lambda x : x['image_id'], get_y = lambda y: y['labels'])
dsets = dblock.datasets(train_df)
dsets.train[0]

In [ ]:
def get_x(r): return path/'images'/r['image_id']
def get_y(r) : return r['labels']
dblock = DataBlock(get_x=get_x, get_y=get_y)
dsets = dblock.datasets(train_df)
dsets.train[0]

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   splitter=RandomSplitter(seed=42),
                   get_x = get_x, get_y = get_y)
dsets = dblock.datasets(train_df)
dsets.train[0]

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   splitter=RandomSplitter(seed=42),
                   get_x = get_x, get_y = get_y,
                  item_tfms=Resize(750),
                  batch_tfms=aug_transforms(size=512, min_scale=0.75))
dls = dblock.dataloaders(train_df)

In [ ]:
dls.show_batch(nrows=3, ncols=7,unique=True)

In [ ]:
learner = cnn_learner(dls,resnet34, 
                      pretrained=True
                      ,metrics=[error_rate,accuracy]).to_fp16()

In [ ]:
learner.fine_tune(2)

In [ ]:
learner = cnn_learner(dls,resnet34, 
                      pretrained=True
                      ,metrics=[error_rate,accuracy]).to_fp16()
lr_min,lr_steep = learner.lr_find()

In [ ]:
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [ ]:
learner = cnn_learner(dls,resnet34, 
                      pretrained=True
                      ,metrics=[error_rate,accuracy]).to_fp16()
learner.fit_one_cycle(7,3e-3)

In [ ]:
learner.unfreeze()

In [ ]:
learner.lr_find()

In [ ]:
learner = cnn_learner(dls,resnet34, 
                      pretrained=True
                      ,metrics=[error_rate,accuracy]).to_fp16()
learner.fit_one_cycle(3, 3e-3)
learner.unfreeze()
learner.fit_one_cycle(4, lr_max=slice(1e-6,1e-4))

In [ ]:
learner.export()

In [ ]:
path.ls()

In [ ]:
learner.show_results()

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)
interp.plot_confusion_matrix(title='Confusion matrix')

In [ ]:
fnames = get_image_files(path/'images')

In [ ]:
test_df = pd.read_csv(r'../input/plant-pathology-2020-fgvc7/test.csv')
test_df['image_id'] = test_df['image_id'] + '.jpg'
test_df['image_id'].astype('str')

In [ ]:
tst_preds = []
tst_dl = dls.test_dl(test_df)
y, _ = learner.tta(dl=tst_dl)
tst_preds.append(y)

In [ ]:
subm = pd.read_csv(path/"sample_submission.csv")

In [ ]:
subm.iloc[:, 1:] = tst_preds[0]

In [ ]:
subm.to_csv("submission3.csv", index=False)